https://github.com/shreyasvedpathak/Tensorflow-Advanced-Techniques-Solutions/blob/main/Course%201/Week4_Assignment.ipynb

In [22]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from tensorflow.keras.models import Model

In [8]:
class MyClass:
    def __init__(self):
        self.var1 = 1
my_obj = MyClass()

In [9]:
my_obj.__dict__ # object's instance variables and values as key value pairs

{'var1': 1}

In [11]:
# using vars function does the same thing
vars(my_obj)

{'var1': 1}

In [12]:


# Add a new instance variable and give it a value
my_obj.var2 = 2

# Calls vars() again to see the object's instance variables
vars(my_obj)



{'var1': 1, 'var2': 2}

In [14]:
# Another way to add a variable which we can use to programmically add instance variables is this
my_obj.__dict__['var3'] = 3
vars(my_obj)['var4'] = 4 # or this

vars(my_obj)

{'var1': 1, 'var2': 2, 'var3': 3, 'var4': 4}

In [15]:
# Use a for loop to increment the index 'i'
for i in range(4,10):
    # Format a string that is var
    vars(my_obj)[f'var{i}'] = i
    
# View the object's instance variables!
vars(my_obj)

{'var1': 1,
 'var2': 2,
 'var3': 3,
 'var4': 4,
 'var5': 5,
 'var6': 6,
 'var7': 7,
 'var8': 8,
 'var9': 9}

In [23]:
# now use the dynamic class var creation method to create a block that has
# an arbitrary number of conv layers
class Block(Model):
    def __init__(self, num_filters, kernel_size, repetitions:int, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.num_filters = num_filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        # define conv layers as instance vars
        for i in range(repetitions):
            self.__dict__[f'conv{i}'] = Conv2D(self.num_filters, self.kernel_size,
                                               padding="same", activation='relu')
        self.max_pooling = MaxPool2D(pool_size=pool_size, strides=strides)
        
        
    def call(self, inputs):
        conv0 = self.__dict__['conv0']
        x = conv0(inputs) # connect the inputs to the conv layer
        
        # connect the rest of the conv layers
        for i in range(1, self.repetitions):
            conv_i = self.__dict__['conv{}'.format(i)]
            x = conv_i(x)
        max_pool  = self.max_pooling(x)
        return max_pool

In [25]:
class MyVGG(Model):
    def __init__(self, num_classes):
        super(MyVGG, self).__init__()
        self.block_a = Block(64, 3, 2)
        self.block_b = Block(128, 3, 2)
        self.block_c = Block(256, 3, 3)
        self.block_d = Block(512, 3, 3)
        self.block_e = Block(512, 3, 3)
        self.flatten = Flatten()
        self.dense = Dense(256, activation = 'relu')
        self.classifier = Dense(num_classes, activation = 'softmax')
        
    def call(self, inputs):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        x = self.flatten(x)
        x = self.dense(x)
        x = self.classifier(x)
        
        return x



In [29]:
ds = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN)
vgg = MyVGG(num_classes=2)
vgg.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

def preprocess(features):
    img = tf.image.resize(features['image'], (244, 244))
    return tf.cast(img, tf.float32)/255., features['label']

ds = ds.map(preprocess).batch(32)

vgg.fit(ds, epochs=10)

Epoch 1/10
727/727 [==============================] - 85s 104ms/step - loss: 0.6823 - accuracy: 0.5694
Epoch 2/10
727/727 [==============================] - 70s 97ms/step - loss: 0.6387 - accuracy: 0.6468
Epoch 3/10
727/727 [==============================] - 70s 97ms/step - loss: 0.6086 - accuracy: 0.6708
Epoch 4/10
727/727 [==============================] - 70s 97ms/step - loss: 0.5940 - accuracy: 0.6809
Epoch 5/10
727/727 [==============================] - 71s 97ms/step - loss: 0.5844 - accuracy: 0.6904
Epoch 6/10
727/727 [==============================] - 70s 97ms/step - loss: 0.5765 - accuracy: 0.6960
Epoch 7/10
727/727 [==============================] - 70s 96ms/step - loss: 0.5695 - accuracy: 0.7014
Epoch 8/10
727/727 [==============================] - 70s 97ms/step - loss: 0.5630 - accuracy: 0.7069
Epoch 9/10
727/727 [==============================] - 70s 97ms/step - loss: 0.5568 - accuracy: 0.7123
Epoch 10/10
727/727 [==============================] - 71s 97ms/step - loss: 0.55

In [31]:
# ds_test = tfds.load('cats_vs_dogs', split=tfds.Split.T) # this is probably the wrong way of getting data but its fine
# ds_test = ds.map(preprocess).batch(32)
vgg.evaluate(ds) # should't use same as the trainig ds but its fine


727/727 [==============================] - 70s 96ms/step - loss: 0.5555 - accuracy: 0.7110


[0.5555102229118347, 0.7109878659248352]